In [10]:
# Importar librerías necesarias
import os
from pymongo import MongoClient
from dotenv import load_dotenv
from datetime import datetime
from bson import ObjectId
import json
import bcrypt

# Cargar variables de entorno
load_dotenv()

# Función para generar hash (igual que en la aplicación)
def hash_password(password: str) -> str:
    """
    Generar hash de contraseña usando bcrypt (igual que en auth_utils.py)
    """
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')

print("✅ Librerías importadas correctamente")
print("✅ Variables de entorno cargadas")
print("✅ Función de hash configurada")


✅ Librerías importadas correctamente
✅ Variables de entorno cargadas
✅ Función de hash configurada


In [11]:
# Configurar conexión a MongoDB
MONGODB_URI = os.getenv('MONGODB_URI')

if not MONGODB_URI:
    print("❌ Error: No se encontró MONGODB_URI en las variables de entorno")
    print("Por favor, asegúrate de tener el archivo .env con la configuración correcta")
else:
    print("✅ URI de MongoDB encontrada")
    # Mostrar URI enmascarada por seguridad
    masked_uri = MONGODB_URI[:20] + "*" * 20 + MONGODB_URI[-20:] if len(MONGODB_URI) > 40 else "*" * len(MONGODB_URI)
    print(f"URI: {masked_uri}")


✅ URI de MongoDB encontrada
URI: mongodb+srv://fitoxm********************ites=true&w=majority


In [12]:
# Conectar a MongoDB
if MONGODB_URI:
    try:
        client = MongoClient(MONGODB_URI)
        
        # Probar la conexión
        client.admin.command('ping')
        print("✅ Conexión exitosa a MongoDB")
        
        # Obtener la base de datos
        db_name = MONGODB_URI.split('/')[-1].split('?')[0]
        db = client[db_name]
        
        print(f"✅ Base de datos conectada: {db_name}")
        
        # Mostrar colecciones existentes
        collections = db.list_collection_names()
        print(f"✅ Colecciones disponibles: {collections}")
        
    except Exception as e:
        print(f"❌ Error al conectar a MongoDB: {e}")
else:
    print("❌ No se puede conectar: MONGODB_URI no está configurada")


✅ Conexión exitosa a MongoDB
✅ Base de datos conectada: lavanderia_db
✅ Colecciones disponibles: ['user_employees', 'user_clients', 'products', 'dryers', 'washers']


In [13]:
# Verificar si ya existe el usuario administrador
user_employees_collection = db['user_employees']

existing_admin = user_employees_collection.find_one({
    "$or": [
        {"username": "admin"},
        {"email": "admin@purimatic.com"}
    ]
})

if existing_admin:
    print("⚠️  Ya existe un usuario administrador en la base de datos:")
    print(f"Username: {existing_admin.get('username')}")
    print(f"Email: {existing_admin.get('email')}")
    print(f"Role: {existing_admin.get('role')}")
    print(f"Is Active: {existing_admin.get('is_active')}")
    print("\n¿Deseas continuar y reemplazar este usuario? (Ejecuta la siguiente celda)")
else:
    print("✅ No se encontró usuario administrador existente. Listo para crear.")


✅ No se encontró usuario administrador existente. Listo para crear.


In [14]:
# Datos del usuario administrador
plain_password = "AdminPurimatic2024"

# Generar hash de contraseña usando el mismo método que la aplicación
password_hash = hash_password(plain_password)

admin_user_data = {
    "_id": ObjectId("6866ced114f19bf5e0dd6bf5"),
    "username": "admin",
    "email": "admin@purimatic.com",
    "password_hash": password_hash,
    "role": "admin",
    "store_id": "store_001",
    "is_active": True,
    "created_at": datetime.fromisoformat("2025-07-03T00:00:00.000Z".replace('Z', '+00:00')),
    "updated_at": datetime.fromisoformat("2025-07-03T00:00:00.000Z".replace('Z', '+00:00'))
}

print("✅ Datos del usuario administrador preparados:")
print(f"Username: {admin_user_data['username']}")
print(f"Email: {admin_user_data['email']}")
print(f"Role: {admin_user_data['role']}")
print(f"Store ID: {admin_user_data['store_id']}")
print(f"Password: {plain_password}")
print(f"Is Active: {admin_user_data['is_active']}")
print(f"Password Hash: {password_hash[:30]}...")
print("\n⚠️  IMPORTANTE: Guarda esta contraseña en un lugar seguro: AdminPurimatic2024")
print("✅ Hash generado usando bcrypt (igual que la aplicación)")


✅ Datos del usuario administrador preparados:
Username: admin
Email: admin@purimatic.com
Role: admin
Store ID: store_001
Password: AdminPurimatic2024
Is Active: True
Password Hash: $2b$12$BnNwJzy9QqgMgtdkyEUrIOh...

⚠️  IMPORTANTE: Guarda esta contraseña en un lugar seguro: AdminPurimatic2024
✅ Hash generado usando bcrypt (igual que la aplicación)


In [15]:
# Crear el usuario administrador
try:
    # Usar upsert para insertar o actualizar
    result = user_employees_collection.replace_one(
        {"_id": admin_user_data["_id"]},
        admin_user_data,
        upsert=True
    )
    
    if result.upserted_id:
        print(f"✅ Usuario administrador creado exitosamente con ID: {result.upserted_id}")
    elif result.modified_count > 0:
        print(f"✅ Usuario administrador actualizado exitosamente")
    else:
        print("ℹ️  No se realizaron cambios (el usuario ya existía con los mismos datos)")
    
except Exception as e:
    print(f"❌ Error al crear el usuario administrador: {e}")


✅ Usuario administrador creado exitosamente con ID: 6866ced114f19bf5e0dd6bf5


In [16]:
# Verificar que el usuario se creó correctamente
try:
    created_user = user_employees_collection.find_one({"username": "admin"})
    
    if created_user:
        print("✅ Usuario administrador verificado exitosamente:")
        print(f"ID: {created_user['_id']}")
        print(f"Username: {created_user['username']}")
        print(f"Email: {created_user['email']}")
        print(f"Role: {created_user['role']}")
        print(f"Store ID: {created_user['store_id']}")
        print(f"Is Active: {created_user['is_active']}")
        print(f"Created At: {created_user['created_at']}")
        print(f"Updated At: {created_user['updated_at']}")
        print(f"Password Hash: {created_user['password_hash'][:20]}...")
        
        print("\n🎉 ¡Usuario administrador listo para usar!")
        print("📋 Credenciales para login:")
        print(f"   Username: {created_user['username']}")
        print(f"   Password: {plain_password}")
        
    else:
        print("❌ Error: No se pudo verificar el usuario creado")
        
except Exception as e:
    print(f"❌ Error al verificar el usuario: {e}")


✅ Usuario administrador verificado exitosamente:
ID: 6866ced114f19bf5e0dd6bf5
Username: admin
Email: admin@purimatic.com
Role: admin
Store ID: store_001
Is Active: True
Created At: 2025-07-03 00:00:00
Updated At: 2025-07-03 00:00:00
Password Hash: $2b$12$BnNwJzy9QqgMg...

🎉 ¡Usuario administrador listo para usar!
📋 Credenciales para login:
   Username: admin
   Password: AdminPurimatic2024


In [17]:
# Verificar que el hash funciona correctamente
def verify_password(password: str, password_hash: str) -> bool:
    """
    Verificar contraseña contra su hash (igual que en auth_utils.py)
    """
    return bcrypt.checkpw(password.encode('utf-8'), password_hash.encode('utf-8'))

# Probar la verificación de contraseña con el hash generado
try:
    test_result = verify_password(plain_password, password_hash)
    if test_result:
        print("✅ Verificación de contraseña exitosa")
        print("✅ El hash es compatible con la aplicación")
    else:
        print("❌ Error en la verificación de contraseña")
except Exception as e:
    print(f"❌ Error al verificar contraseña: {e}")

# Información adicional sobre la base de datos
try:
    # Contar documentos en las colecciones principales
    stats = {
        "user_employees": user_employees_collection.count_documents({}),
        "user_clients": db.user_clients.count_documents({}) if "user_clients" in collections else 0,
        "products": db.products.count_documents({}) if "products" in collections else 0,
        "washers": db.washers.count_documents({}) if "washers" in collections else 0,
        "dryers": db.dryers.count_documents({}) if "dryers" in collections else 0,
        "stores": db.stores.count_documents({}) if "stores" in collections else 0
    }
    
    print("\n📊 Estadísticas de la base de datos:")
    for collection, count in stats.items():
        print(f"   {collection}: {count} documentos")
    
    # Verificar si existe la tienda store_001
    if "stores" in collections:
        store_001 = db.stores.find_one({"_id": "store_001"})
        if store_001:
            print(f"\n✅ Tienda store_001 encontrada: {store_001.get('nombre', 'Sin nombre')}")
        else:
            print("\n⚠️  Tienda store_001 no encontrada. Podrías necesitar crear los datos de la tienda.")
            
except Exception as e:
    print(f"❌ Error al obtener estadísticas: {e}")


✅ Verificación de contraseña exitosa
✅ El hash es compatible con la aplicación

📊 Estadísticas de la base de datos:
   user_employees: 1 documentos
   user_clients: 0 documentos
   products: 0 documentos
   washers: 0 documentos
   dryers: 0 documentos
   stores: 0 documentos


In [18]:
# Cerrar la conexión
client.close()
print("✅ Conexión a MongoDB cerrada")
print("\n🎯 Resumen:")
print("   - Usuario administrador creado/actualizado")
print("   - Credenciales listas para usar")
print("   - Base de datos verificada")
print("\n🚀 ¡Ahora puedes hacer login en tu aplicación con:")
print("   Username: admin")
print("   Password: AdminPurimatic2024")


✅ Conexión a MongoDB cerrada

🎯 Resumen:
   - Usuario administrador creado/actualizado
   - Credenciales listas para usar
   - Base de datos verificada

🚀 ¡Ahora puedes hacer login en tu aplicación con:
   Username: admin
   Password: AdminPurimatic2024


In [19]:
# 🧪 Demostración de las funciones de hash (opcional)
print("🧪 Demostración de las funciones de hash:")
print("=====================================")

# Generar diferentes hashes para la misma contraseña
password_test = "AdminPurimatic2024"
hash1 = hash_password(password_test)
hash2 = hash_password(password_test)

print(f"Contraseña: {password_test}")
print(f"Hash 1: {hash1}")
print(f"Hash 2: {hash2}")
print(f"¿Son iguales? {hash1 == hash2}")

print("\n🔍 Verificando ambos hashes:")
print(f"Hash 1 verifica correctamente: {verify_password(password_test, hash1)}")
print(f"Hash 2 verifica correctamente: {verify_password(password_test, hash2)}")

print("\n✨ Conclusión:")
print("- Cada hash es único (por el salt aleatorio)")
print("- Pero ambos verifican correctamente la misma contraseña")
print("- Esto es normal y seguro con bcrypt")
print("- Tu aplicación funcionará correctamente con cualquier hash generado")


🧪 Demostración de las funciones de hash:
Contraseña: AdminPurimatic2024
Hash 1: $2b$12$mrz4fXpOi/ybo3dng0wqf.3DkMJQp1BvhQh3VSZ3crfS189bxtZe.
Hash 2: $2b$12$h4ev547bBiog4HP.4MOfH.7NMgSOVCoWuhnjc5O5xHldEFR0uDu/y
¿Son iguales? False

🔍 Verificando ambos hashes:
Hash 1 verifica correctamente: True
Hash 2 verifica correctamente: True

✨ Conclusión:
- Cada hash es único (por el salt aleatorio)
- Pero ambos verifican correctamente la misma contraseña
- Esto es normal y seguro con bcrypt
- Tu aplicación funcionará correctamente con cualquier hash generado
